 🦉Cuando subes un archivo manualmente a Colab, este se
    almacena en el entorno temporal
 (que se borra al cerrar la sesión). 📁 debes moverlo a tu Google Drive para conservarlo
  ya que en cada actualización se pierde los que subes y molesto actualizar.

Sube el archivo PDF usando el botón de upload de Colab, el archivo estára en /content/

Usa el nombre exacto en este caso es (CEPL-Aspirantes-Insc-0712-1230.pdf)

Debes verificar que quedó guardado en tu carpeta  /MyDrive:

In [22]:
from google.colab import drive
import shutil
import os

# Montar Google Drive (con force_remount para evitar advertencias)
drive.mount('/content/drive', force_remount=True)

# Verificar archivo en Colab
ruta_origen = '/content/CEPL-Aspirantes-Insc-0712-1230.pdf'
ruta_destino = '/content/drive/MyDrive/ConcursoJueces/'

# ---------------------------
# Paso 1: Verificar origen
# ---------------------------
print("🔍 Contenido de /content/ (Colab):")
print(os.listdir('/content'))  # Mostrar archivos en Colab

if os.path.exists(ruta_origen):
    print(f"\n✅ Archivo encontrado en Colab: {ruta_origen}")

    # ---------------------------
    # Paso 2: Crear carpeta ConcursoJueces si no existe
    # ---------------------------
    os.makedirs(ruta_destino, exist_ok=True)

    # ---------------------------
    # Paso 3: Copiar archivo
    # ---------------------------
    try:
        shutil.copy(ruta_origen, ruta_destino)
        print(f"\n✅ Archivo copiado a Drive: {ruta_destino}")

        # Verificación final
        if os.path.exists(os.path.join(ruta_destino, os.path.basename(ruta_origen))):
            print(f"\n🔥 Archivo confirmado en Drive: {ruta_destino}")
        else:
            print("\n⚠️ Error: La copia no tuvo efecto")

    except Exception as e:
        print(f"\n❌ Error al copiar: {str(e)}")

else:
    print(f"\n❌ Archivo NO encontrado en Colab: {ruta_origen}")
    print("Posibles causas:")
    print("1. El nombre del archivo no coincide (ver mayúsculas y extensión .pdf)")
    print("2. No se subió manualmente a Colab")
    print("3. Se subió a otra ruta (no a /content/)")

Mounted at /content/drive
🔍 Contenido de /content/ (Colab):
['.config', 'CEPL-Aspirantes-Insc-0712-1230.pdf', 'drive', '.ipynb_checkpoints', 'sample_data']

✅ Archivo encontrado en Colab: /content/CEPL-Aspirantes-Insc-0712-1230.pdf

✅ Archivo copiado a Drive: /content/drive/MyDrive/ConcursoJueces/

🔥 Archivo confirmado en Drive: /content/drive/MyDrive/ConcursoJueces/


🦉 Intala la libreria para trabajar con PDF´s

In [24]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 44.7 MB/s eta 0:00:00
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20240706
    Uninstalling pdfminer.six-20240706:
      Successfully uninstalled pdfminer.six-20240706
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
camelot-py 1.0.0 requires pdfminer-six>=20240706, but you have pdfminer-six 20231228 which is incompatible.


🧙  Trasforma la tabla 📑 en un CSV 📄

In [25]:
from google.colab import drive
import pandas as pd
import pdfplumber
import os

# Montar Google Drive (solo necesitas hacerlo una vez por sesión)
drive.mount('/content/drive')

# Leer el PDF
with pdfplumber.open('CEPL-Aspirantes-Insc-0712-1230.pdf') as pdf:
    print(f"El PDF tiene {len(pdf.pages)} páginas.")

    # Lista para almacenar los datos extraídos
    all_rows = []

    # Procesar cada página
    for page_num, page in enumerate(pdf.pages, start=1):
        # Extraer el texto de la página
        page_text = page.extract_text()

        # Dividir el texto en líneas
        lines = page_text.split('\n')

        # Procesar cada línea
        for line in lines:
            # Dividir la línea en palabras
            parts = line.split()
            if len(parts) >= 3 and parts[0].isdigit():
                consecutivo = parts[0]

                # Identificar el inicio del cargo mediante palabras clave o patrones
                cargo_keywords = [
                    "Ministras", "Ministros", "Magistradas", "Magistrados",
                    "Tribunal", "Disciplina", "Justicia", "Sala",
                    "Superior", "Electoral", "Poder", "Federación",
                    "Circuito", "validación","En","Juezas"
                ]

                # Buscar el índice donde comienza el cargo
                cargo_index = next((i for i, word in enumerate(parts[1:], start=1) if word in cargo_keywords), len(parts))

                # Separar el nombre y el cargo
                nombre = ' '.join(parts[1:cargo_index])  # Todo antes del cargo
                cargo = ' '.join(parts[cargo_index:])  # Todo a partir del inicio del cargo

                # Agregar los datos extraídos a la lista
                all_rows.append([page_num, consecutivo, nombre.strip(), cargo.strip()])

# Crear el DataFrame y exportar a CSV
df = pd.DataFrame(all_rows, columns=['Página', 'Consecutivo', 'Nombre', 'Cargo'])

# Ruta de guardado en Drive
save_dir = '/content/drive/MyDrive/ConcursoJueces/'  # Puedes cambiar esta ruta
os.makedirs(save_dir, exist_ok=True)  # Crear directorio si no existe

output_path = os.path.join(save_dir, 'pruebauno.csv')
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n✅ Archivo guardado en Drive: {output_path}")
print(f"Total de registros: {len(df)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
El PDF tiene 149 páginas.

✅ Archivo guardado en Drive: /content/drive/MyDrive/ConcursoJueces/pruebauno.csv
Total de registros: 10959


Pubica el CSV de tu drive en googleSheet para revisarlo 🔗



  | CONSECUTIVO | NOMBRE | Cargo   |
|-------------|--------|---------|
| 01          | MPFJ   | JUECES  |
| 02          | ABC    | MAGISTRADO |
| 03          | XYZ    | FISCAL  |

In [33]:
from google.colab import drive
from googleapiclient.discovery import build
from oauth2client.client import GoogleCredentials

# Montar Google Drive
drive.mount('/content/drive')

# Autenticación para usar la API de Google Drive
credentials = GoogleCredentials.get_application_default()
service = build('drive', 'v3', credentials=credentials)

# ID del archivo ya existente en Google Sheets
file_id = '1cELLJBG91TPimvarl84MZlpA0jXrdPeieBVvMmqhMzo'

# Cambiar permisos del archivo para que sea público
permissions = {
    'type': 'anyone',  # Permitir acceso a cualquiera con el enlace
    'role': 'reader'   # Solo lectura
}
service.permissions().create(fileId=file_id, body=permissions).execute()

# Generar enlace público para Google Sheets
sheets_link = f"https://docs.google.com/spreadsheets/d/{file_id}/edit"

print(f"El archivo ahora es público. Accede aquí: {sheets_link}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
El archivo ahora es público. Accede aquí: https://docs.google.com/spreadsheets/d/1cELLJBG91TPimvarl84MZlpA0jXrdPeieBVvMmqhMzo/edit


s

In [1]:
import pandas as pd
import re
from pathlib import Path
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

def validar_nombre(nombre):
    nombre = str(nombre)
    problemas = []
    if re.search(r'\d', nombre):
        problemas.append("contiene_números")
    if '  ' in nombre:
        problemas.append("espacios_múltiples")
    if nombre.strip() != nombre:
        problemas.append("espacios_extremos")
    return problemas if problemas else None

# Configuración de rutas en Google Drive
ARCHIVO_ORIGINAL = "/content/drive/MyDrive/ConcursoJueces/pruebauno.csv"
RUTA_REPORTE = "/content/drive/MyDrive/ConcursoJueces/reportes/problemas_nombre.csv"

# 1. Leer archivo original
try:
    listaOriginal = pd.read_csv(ARCHIVO_ORIGINAL, dtype=str)
    print(f"✅ Archivo leído: {Path(ARCHIVO_ORIGINAL).resolve()}")
except FileNotFoundError as e:
    print(f"❌ Error crítico: {str(e)}")
    print(f"Verifica que el archivo existe en esta ubicación:\n{Path(ARCHIVO_ORIGINAL).resolve()}")
    exit()

# 2. Crear copia limpia
Copia_lista = listaOriginal.copy()
Copia_lista['Nombre'] = Copia_lista['Nombre'].str.strip()

# 3. Detectar registros problemáticos (usando el original)
falsos_registros = listaOriginal[
    listaOriginal['Nombre'].apply(
        lambda x: bool(validar_nombre(x))
    )
].copy()

# 4. Guardar reporte con verificación
try:
    # Crear estructura de directorios
    Path(RUTA_REPORTE).parent.mkdir(parents=True, exist_ok=True)

    # Guardar CSV con codificación adecuada
    falsos_registros.to_csv(RUTA_REPORTE, index=False, encoding='utf-8-sig')

    # Verificación post-guardado
    if Path(RUTA_REPORTE).exists():
        print(f"\n💾 Reporte guardado en: {Path(RUTA_REPORTE).resolve()}")
        print(f"📄 Tamaño del archivo: {Path(RUTA_REPORTE).stat().st_size} bytes")
        print(f"📝 Registros en reporte: {len(falsos_registros)}")
    else:
        raise Exception("El archivo no se creó debido a un error desconocido")

except Exception as e:
    print(f"\n❌ Error al guardar el reporte: {str(e)}")
    print(f"Ruta intentada: {Path(RUTA_REPORTE).resolve()}")
    print("Verifica los permisos de escritura en Google Drive")

# 5. Mostrar resumen detallado
print("\n" + "═"*50)
print("RESUMEN EJECUCIÓN".center(50))
print("═"*50)
print(f"📂 Archivo original: {Path(ARCHIVO_ORIGINAL).name}")
print(f"📦 Total registros: {len(listaOriginal):,}")
print(f"🚨 Problemas detectados: {len(falsos_registros):,}")
print(f"🧹 Registros limpios: {len(Copia_lista):,}")
print("═"*50)

# Mostrar muestra de registros problemáticos
if not falsos_registros.empty:
    print("\n🔍 Primeros 3 registros con problemas:")
    print(falsos_registros[['Página', 'Consecutivo', 'Nombre', 'Cargo']].head(3).to_markdown(index=False))
else:
    print("\n🎉 ¡No se encontraron registros problemáticos!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Archivo leído: /content/drive/MyDrive/ConcursoJueces/pruebauno.csv

💾 Reporte guardado en: /content/drive/MyDrive/ConcursoJueces/reportes/problemas_nombre.csv
📄 Tamaño del archivo: 450 bytes
📝 Registros en reporte: 5

══════════════════════════════════════════════════
                RESUMEN EJECUCIÓN                 
══════════════════════════════════════════════════
📂 Archivo original: pruebauno.csv
📦 Total registros: 10,959
🚨 Problemas detectados: 5
🧹 Registros limpios: 10,959
══════════════════════════════════════════════════

🔍 Primeros 3 registros con problemas:
|   Página |   Consecutivo | Nombre                   | Cargo                                                           |
|---------:|--------------:|:-------------------------|:----------------------------------------------------------------|
|       42 |          3049 | LOPEZ ARVIZU GUADALUP